In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

use_latex = True # Set to True to use LaTeX for text rendering

In [11]:
if use_latex:
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": 'Computer Modern Roman'
    })

In [16]:
# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M = 5.972e24  # Mass of the Earth (kg)
mu = G * M

# Convert to Cartesian coordinates
def convert_to_cartesian(e, a, i, O, w, nu, t, t0, object_scale=1.0):
    # Calculate eccentric anomaly from true anomaly
    E = 2 * np.arctan2(np.sqrt(1 - e) * np.sin(nu / 2), np.sqrt(1 + e) * np.cos(nu / 2))

    # Calculate mean anomaly from eccentric anomaly
    Mt = E - e * np.sin(E)

    # Calculate the orbital period
    T = 2 * np.pi * np.sqrt(a**3 / mu)

    # Calculate the time increment
    deltaT = (T / 60) * (t - t0)
    Mt = Mt + deltaT * np.sqrt(mu / a**3)

    # Calculate eccentric anomaly using Newton's method
    F = E - e * np.sin(E) - Mt
    j = 0
    max_iter = 30
    delta = 1e-6
    while abs(F) > delta and j < max_iter:
        E = E - F / (1 - e * np.cos(E))
        F = E - e * np.sin(E) - Mt
        j += 1

    # Calculate true anomaly from updated eccentric anomaly
    nu = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2), np.sqrt(1 - e) * np.cos(E / 2))

    # Distance to central body
    rc = a * (1 - e * np.cos(E))

    # Position in orbital plane
    o = np.array([rc * np.cos(nu), rc * np.sin(nu), 0])

    # Transform to Cartesian coordinates
    rx = o[0]
    ry = o[1]
    rz = o[2]

    rx = (o[0] * (np.cos(w) * np.cos(O) - np.sin(w) * np.cos(i) * np.sin(O)) -
          o[1] * (np.sin(w) * np.cos(O) + np.cos(w) * np.cos(i) * np.sin(O)))
    ry = (o[0] * (np.cos(w) * np.sin(O) + np.sin(w) * np.cos(i) * np.cos(O)) +
          o[1] * (np.cos(w) * np.cos(i) * np.cos(O) - np.sin(w) * np.sin(O)))
    rz = (o[0] * (np.sin(w) * np.sin(i)) + o[1] * (np.cos(w) * np.sin(i)))

    # Position vector
    r = np.array([rx, ry, rz])

    return r / object_scale

# Convert to Keplerian elements
def convert_to_kepler_elements(r, v):
    # Calculate angular momentum vector
    h = np.cross(r, v)
    n = np.cross([0, 0, 1], h)

    # Calculate eccentricity vector
    evec = ((np.linalg.norm(v)**2 - mu / np.linalg.norm(r)) * np.array(r) - np.dot(r, v) * np.array(v)) / mu
    emag = np.linalg.norm(evec)

    # Specific mechanical energy
    E = np.linalg.norm(v)**2 / 2 - mu / np.linalg.norm(r)

    # Semi-major axis
    if emag != 1:
        a = -mu / (2 * E)
        p = a * (1 - emag**2)
    else:
        a = np.inf
        p = np.linalg.norm(h)**2 / mu

    # Inclination
    inc = np.arccos(h[2] / np.linalg.norm(h))

    # Longitude of ascending node
    if inc == 0 or inc == np.pi:
        O = 0
    else:
        O = np.arccos(n[0] / np.linalg.norm(n))
    if n[1] < 0:
        O = 2 * np.pi - O

    # Argument of periapsis
    if emag == 0:
        w = 0
    else:
        if inc == 0 or inc == np.pi:
            w = np.arctan2(evec[1], evec[0])
        else:
            w = np.arccos(np.dot(n, evec) / (np.linalg.norm(n) * emag))
    if evec[2] < 0:
        w = 2 * np.pi - w

    # True anomaly
    if emag == 0:
        if inc == 0 or inc == np.pi:
            nu = np.arccos(r[2] / np.linalg.norm(r))
        else:
            nu = np.arccos(np.dot(n, r) / (np.linalg.norm(n) * np.linalg.norm(r)))
    else:
        nu = np.arccos(np.dot(evec, r) / (emag * np.linalg.norm(r)))
        if np.dot(r, v) < 0:
            nu = 2 * np.pi - nu

    # Convert angles from radians to degrees
    inc = np.degrees(inc)
    O = np.degrees(O)
    w = np.degrees(w)
    nu = np.degrees(nu)

    return {
        "semi_major_axis": a,
        "semi_latus_rectum": p,
        "eccentricity": emag,
        "inclination": inc,
        "longitude_of_ascending_node": O,
        "argument_of_periapsis": w,
        "true_anomaly": nu
    }


In [17]:
# Example usage
r = np.array([7000, 0, 0])  # Position vector (example)
v = np.array([0, 7.5, 0])   # Velocity vector (example)

kepler_elements = convert_to_kepler_elements(r, v)
print(kepler_elements)

e = kepler_elements['eccentricity']
a = kepler_elements['semi_major_axis']
i = np.radians(kepler_elements['inclination'])
O = np.radians(kepler_elements['longitude_of_ascending_node'])
w = np.radians(kepler_elements['argument_of_periapsis'])
nu = np.radians(kepler_elements['true_anomaly'])
t = 0  # Current time
t0 = 0  # Reference time

cartesian_position = convert_to_cartesian(e, a, i, O, w, nu, t, t0)
print(cartesian_position)

{'semi_major_axis': 3500.0000017287534, 'semi_latus_rectum': 6.915014675397608e-06, 'eccentricity': 0.9999999990121408, 'inclination': 0.0, 'longitude_of_ascending_node': 0.0, 'argument_of_periapsis': -180.0, 'true_anomaly': 180.0}
[7000.    0.    0.]
